# 对比DenseNet121 与 resNet50 进行分类的效果 两者差别不大，densenet121效果更佳

数据集下载地址 http://download.tensorflow.org/example_images/flower_photos.tgz

In [ ]:
# --coding:utf-8--
import os
import sys
import glob
import matplotlib.pyplot as plt
 
from tensorflow.keras import __version__
from tensorflow.keras.applications.densenet import DenseNet121,preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
 
def get_nb_files(directory):
  """Get number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt
 
 
# 数据准备
IM_WIDTH, IM_HEIGHT = 224, 224 #densenet指定的图片尺寸
 
 
dir = '../flower_photosM'
train_dir = dir+'/train'  # 训练集数据路径
val_dir = dir+'/validate' # 验证集数据
nb_classes= 5
nb_epoch = 10
batch_size = 8
 
nb_train_samples = get_nb_files(train_dir)      # 训练样本个数
nb_classes = len(glob.glob(train_dir + "/*"))  # 分类数
nb_val_samples = get_nb_files(val_dir)       #验证集样本个数
nb_epoch = int(nb_epoch)                # epoch数量
batch_size = int(batch_size)           

In [ ]:
#　图片生成器

train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)
test_datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)
 
# 训练数据与测试数据
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(IM_WIDTH, IM_HEIGHT),
batch_size=batch_size,class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
val_dir,
target_size=(IM_WIDTH, IM_HEIGHT),
batch_size=batch_size,class_mode='categorical')
 
# 添加新层
def add_new_last_layer(base_model, nb_classes):
  """
  添加最后的层
  输入
  base_model和分类数量
  输出
  新的keras的model
  """
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
  model = Model(base_model.input, predictions)
  return model
 
# #搭建模型
model = DenseNet121(include_top=False)
# # model = ResNet50(include_top=False)
model = add_new_last_layer(model, nb_classes)


In [ ]:
model.compile(optimizer=SGD(lr=0.001, momentum=0.9,decay=0.0001,nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])

#开始训练
history_ft = model.fit(
train_generator,
steps_per_epoch=int(nb_train_samples/batch_size),
epochs=nb_epoch,
# callbacks=[checkpoint],
validation_data=validation_generator,
validation_steps=int(nb_val_samples/batch_size))
# densnet121  耗时11.7min     loss: 0.0109 - accuracy: 0.9984 - val_loss: 0.2603 - val_accuracy: 0.9366
# resnet50    耗时12.85min  loss: 0.0180 - accuracy: 0.9956 - val_loss: 0.3063 - val_accuracy: 0.9195
# efficentNetB0  10 epoch   loss: 2.7590 - acc: 0.4189 - val_loss: 1.7219 - val_acc: 0.4682
# efficentNetB5  10 epoch  20min loss: 5.0546 - accuracy: 0.6882 - val_loss: 7.6975 - val_accuracy: 0.4983

In [ ]:
def writeFile(history,filename):
    f = open(filename,'w')
    f.write(str(history.history))
    f.close()

In [ ]:
writeFile(history_tl,'history_lt.txt')

In [ ]:
f = open('history_res.txt','r')
M = f.readlines()
type(eval(M[0]))

# 可视化

In [6]:
import plotly.graph_objects as go
import numpy as np
def showlossAndaccu(fig,fig2,history,netName):
    # Create traces
    # fig = go.Figure()
   
    acc = history['accuracy']
    val_acc = history['val_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(len(acc))
    fig.add_trace(go.Scatter(x=np.array(epochs), y=acc,mode='lines+markers',name=netName+"_acc"))
    fig.add_trace(go.Scatter(x=np.array(epochs), y=val_acc,mode='lines+markers',name=netName+"_val_acc"))
    # fig.show()

    # fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=np.array(epochs), y=loss,mode='lines+markers',name=netName+"_loss"))
    fig2.add_trace(go.Scatter(x=np.array(epochs), y=val_loss,mode='lines+markers',name=netName+"_val_loss"))
    # fig2.show()
    return fig,fig2

In [ ]:
fig = go.Figure()
fig2 = go.Figure()
fig,fig2 = showlossAndaccu(fig,fig2,history_res,"resNet50")
fig,fig2 = showlossAndaccu(fig,fig2,history_dense,"denseNet121")
fig.show()
fig2.show()

# 使用efficientNet进行分类

In [ ]:
from keras_efficientnets import EfficientNetB5 as Net   # Import efficientnet and load the conv base model
# Hyper parameters 超参数
batch_size = 8

width = 224
height = 224
epochs = 10
NUM_TRAIN = 2500

NUM_TEST = 1170
dropout_rate = 0.2
input_shape = (height, width, 3)

In [ ]:
train_dir = 'flower_photosM/train'
validation_dir = 'flower_photosM/validate'

In [ ]:
import os
import sys
import glob
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint

train_datagen = ImageDataGenerator(          
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
from tensorflow import keras
class_num = 5
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# conv_base.summary()

In [ ]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
if dropout_rate > 0:
    model.add(layers.Dropout(dropout_rate, name="dropout_out"))
model.add(layers.Dense(class_num, activation='softmax', name="fc_out")) 

# 输出网络模型参数
# model.summary()  

# 冻结卷积层不参与训练
conv_base.trainable = False
model.compile(optimizer=SGD(lr=0.001, momentum=0.9,decay=0.0001,nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=2e-5),metrics=['acc'])

In [ ]:
history_tl = model.fit(       
      train_generator,
      steps_per_epoch= NUM_TRAIN //batch_size,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps= NUM_TEST //batch_size,
      verbose=1,
      use_multiprocessing=True,
      workers=0
)
# efficentNetB0  10 epoch
# loss: 2.7590 - acc: 0.4189 - val_loss: 1.7219 - val_acc: 0.4682
# efficentNetB5  10 epoch  20min
# loss: 5.0546 - accuracy: 0.6882 - val_loss: 7.6975 - val_accuracy: 0.4983
# 训练后的模型文件保存目录
# model.save('./output_model_file/my_model.h5')

In [4]:
history_dense = eval(open('history_dense.txt','r').readlines()[0])
history_res = eval(open('history_res.txt','r').readlines()[0])
history_tl = eval(open('history_lt.txt','r').readlines()[0])

In [7]:
fig = go.Figure()
fig2 = go.Figure()
fig,fig2 = showlossAndaccu(fig,fig2,history_tl,"efficientB5")
fig,fig2 = showlossAndaccu(fig,fig2,history_dense,"denseNet121")
fig,fig2 = showlossAndaccu(fig,fig2,history_res,"resNet50")
fig.show()
fig2.show()